In [15]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
import nibabel as nib
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from skimage.morphology import binary_dilation, binary_erosion
from skimage.transform import resize

def resize_image(image, target_shape):
    # Resize the image using skimage.resize to match the target shape
    resized_image = resize(image, target_shape, mode='constant', preserve_range=True)
    return resized_image


In [16]:
# Define Dice coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# Define Jaccard index
def jaccard_index(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection + 1)

# Define Boundary F1 Score
def boundary_f1_score(y_true, y_pred, dilation_radius=1):
    def get_boundary(mask, dilation_radius):
        dilation = binary_dilation(mask, selem=np.ones((dilation_radius, dilation_radius, dilation_radius)))
        erosion = binary_erosion(mask, selem=np.ones((dilation_radius, dilation_radius, dilation_radius)))
        return np.logical_xor(dilation, erosion)

    y_true_boundary = get_boundary(y_true.astype(np.bool_), dilation_radius)
    y_pred_boundary = get_boundary(y_pred.astype(np.bool_), dilation_radius)

    intersection = np.sum(y_true_boundary & y_pred_boundary)
    precision = intersection / (np.sum(y_pred_boundary) + 1e-6)
    recall = intersection / (np.sum(y_true_boundary) + 1e-6)
    return 2 * precision * recall / (precision + recall + 1e-6)

In [17]:
# Load the saved model
model_path = 'unet3d_best_model.h5'
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

model = load_model(model_path, custom_objects={'dice_coefficient': dice_coefficient, 'jaccard_index': jaccard_index, 'boundary_f1_score' : boundary_f1_score})

In [18]:
def load_nifti_memmap(img_path):
    
    
    img = nib.load(img_path)
    data = img.get_fdata(dtype=np.float32, caching='unchanged')  # Memory-mapped array
    affine = img.affine
    header = img.header
    return data, affine, header

def resize_volume(volume, target_shape):
    """
    Resizes a 3D volume to the target shape using interpolation.

    Args:
        volume (numpy.ndarray): Input 3D volume.
        target_shape (tuple): Target shape (depth, height, width).

    Returns:
        numpy.ndarray: Resized 3D volume.
    """
    factors = (
        target_shape[0] / volume.shape[0],
        target_shape[1] / volume.shape[1],
        target_shape[2] / volume.shape[2]
    )
    resized_volume = zoom(volume, factors, order=1)  # Order=1 for linear interpolation
    return resized_volume

def resize_image(image, target_shape):
    # Resize the image to match the target shape (height, width)
    return resize(image, target_shape, preserve_range=True, anti_aliasing=True)

def pad_or_crop_volume(volume, target_shape):
    current_shape = volume.shape
    
    # Calculate padding width for each dimension
    pad_width = [(0, max(target_shape[i] - current_shape[i], 0)) for i in range(3)]
    
    # Pad the volume to the target shape
    volume = np.pad(volume, pad_width, mode='constant', constant_values=0)
    
    # Calculate cropping dimensions for each dimension
    crop_start = [(volume.shape[i] - target_shape[i]) // 2 for i in range(3)]
    crop_end = [crop_start[i] + target_shape[i] for i in range(3)]
    
    # Crop the volume to the target shape
    slices = [slice(crop_start[i], crop_end[i]) for i in range(3)]
    volume = volume[slices[0], slices[1], slices[2]]
    
    return volume





In [62]:
# Define the testing generator
#os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
#tf.config.LogicalDeviceConfiguration(memory_limit=10240000)
def data_generator(file_list, test_data_path, test_mask_path, batch_size, target_shape=None):
    while True:
        for start in range(0, len(file_list), batch_size):
            end = min(start + batch_size, len(file_list))
            batch_files = file_list[start:end]
            
            X_batch = []
            y_batch = []
            
            for filename in batch_files:
                img_path = os.path.join(test_data_path, filename)
                corresponding_mask_path = os.path.join(test_mask_path, filename)
                
                # Load NIfTI data (assumes a function `load_nifti_memmap` is already defined)
                image, _, _ = load_nifti_memmap(img_path)
                mask, _, _ = load_nifti_memmap(corresponding_mask_path)
                
                if target_shape:
                    image = resize_volume(image, target_shape)
                    mask = resize_volume(mask, target_shape)
                
                X_batch.append(image)
                y_batch.append(mask)
            
            X_batch = np.array(X_batch)[..., np.newaxis]
            y_batch = np.array(y_batch)[..., np.newaxis]
            
            yield X_batch, y_batch

# Paths for testing data and masks
test_data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Test/imageTs'
test_mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Test/LabelTs'


# Filter for the specific patient's file (example: patient ID starts with "patient001")
test_files = [filename for filename in os.listdir(test_data_path) if "liver_127.nii.gz" in filename]

# Define batch size and target shape
batch_size = 1
input_shape = (128, 128, 800, 1)

# Create testing generator
test_generator = data_generator(test_files, test_data_path, test_mask_path, batch_size, target_shape=input_shape[:3])

# Calculate steps for testing
test_steps = 1

# Perform evaluation on the test set
test_loss, test_accuracy, test_dice, test_jaccard = model.evaluate(test_generator, steps=test_steps)

# Print the test metrics
print("\nTest Metrics:")
print(f"Test file: {test_files}")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Dice Coefficient: {test_dice}")
print(f"Test Jaccard Index: {test_jaccard}")


1/1 [==============================] - 1s 658ms/step - loss: 0.0314 - accuracy: 0.9901 - dice_coefficient: 0.5630 - jaccard_index: 0.3918

Test Metrics:
Test file: ['liver_127.nii.gz']
Test Loss: 0.03138776123523712
Test Accuracy: 0.9900537133216858
Test Dice Coefficient: 0.563030481338501
Test Jaccard Index: 0.39181894063949585


In [63]:
# Save test metrics to a file
test_metrics = {
    "Test file": test_files,
    "Test Loss": test_loss,
    "Test Accuracy": test_accuracy,
    "Test Dice Coefficient": test_dice,
    "Test Jaccard Index": test_jaccard,
    
}
# Define the file path
metrics_file = 'test_metrics.csv'

# Check if the file exists
file_exists = os.path.isfile(metrics_file)

# Convert metrics to a DataFrame
metrics_df = pd.DataFrame([test_metrics])

# Append metrics to the file
metrics_df.to_csv(metrics_file, mode='a', index=False, header=not file_exists)

print(f"\nTest Metrics appended to '{metrics_file}'.")


Test Metrics appended to 'test_metrics.csv'.


In [64]:
def display_and_save_results_for_one_patient(model, patient_filename, test_data_path, test_mask_path, save_dir):
    target_shape = (128, 128, 800, 1)  # Fixed shape for all volumes

    # Construct file paths for the image and mask
    image_file_path = os.path.join(test_data_path, patient_filename)
    mask_file_path = os.path.join(test_mask_path, patient_filename)

    print(f"Processing file: {patient_filename}")

    try:
        # Load image and mask data from .nii files
        image = nib.load(image_file_path).get_fdata(dtype=np.float32)
        mask = nib.load(mask_file_path).get_fdata(dtype=np.float32)

        # Resize image and mask to expected shape
        image = resize_image(image, target_shape[:2])
        mask = resize_image(mask, target_shape[:2])

        # Pad or crop image and mask to match target depth
        image = pad_or_crop_volume(image, target_shape)
        mask = pad_or_crop_volume(mask, target_shape)

        # Preprocess image and mask
        image = image[..., np.newaxis]
        mask = mask[..., np.newaxis]

        # Get prediction
        prediction = model.predict(np.expand_dims(image, axis=0))[0, ..., 0]

        # Ensure save directory exists
        os.makedirs(save_dir, exist_ok=True)

        # Save each slice of the image, mask, and prediction
        for z in range(target_shape[2]):
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))

            # Subplot 1: Original Image Slice
            axs[0].imshow(image[:, :, z, 0], cmap='gray')
            axs[0].set_title(f"Image Slice {z}")
            axs[0].axis('off')

            # Subplot 2: Ground Truth Mask Slice
            axs[1].imshow(mask[:, :, z, 0], cmap='gray')
            axs[1].set_title(f"Mask Slice {z}")
            axs[1].axis('off')

            # Subplot 3: Prediction Slice
            axs[2].imshow(prediction[:, :, z], cmap='gray')
            axs[2].set_title(f"Prediction Slice {z}")
            axs[2].axis('off')

            plt.tight_layout()

            # Save the plot
            save_path = os.path.join(save_dir, f"{patient_filename}_slice_{z}.png")
            plt.savefig(save_path, dpi=300)
            plt.close(fig)  # Close the figure to free memory

            print(f"Saved slice {z} to {save_path}")

    except Exception as e:
        print(f"Error processing file {patient_filename}: {e}")

# Specify the single patient's filename and save directory
patient_filename = "liver_127.nii.gz"  # Replace with the desired filename
save_dir = "/home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127"  # Replace with the desired save directory

# Call the function to process and save the results
display_and_save_results_for_one_patient(model, patient_filename, test_data_path, test_mask_path, save_dir)


Processing file: liver_127.nii.gz
1/1 [==============================] - 0s 43ms/step
Saved slice 0 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_0.png
Saved slice 1 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_1.png
Saved slice 2 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_2.png
Saved slice 3 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_3.png
Saved slice 4 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_4.png
Saved slice 5 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_5.png
Saved slice 6 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_6.png
Saved slice 7 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_7.png
Saved slice 8 to /home/icmr/Documents/MSD_LIVER_OUTPUT/PORTALPHASE/127/liver_127.nii.gz_slice_8.png
Saved slice 9 